In [1]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast
import joblib
import os
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizerFast, get_linear_schedule_with_warmup
from datasets import Dataset
from tqdm import tqdm
import warnings

import pandas as pd
import torch
from transformers import DistilBertTokenizerFast
import joblib
import os
import numpy as np
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Suppress warnings if needed
warnings.filterwarnings('ignore')

# Load model and resources
model_dir = r"U:\N\save"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_dir)
label_enc = joblib.load(os.path.join(model_dir, "label_encoder.pkl"))
status_enc = joblib.load(os.path.join(model_dir, "status_encoder.pkl"))
status_map = {0: 'accepted', 1: 'accepted', 2: 'accepted',
              3: 'pending', 4: 'pending',
              5: 'blocked', 6: 'blocked', 7: 'blocked'}

# Model definition (must match training architecture)
class ChatModerationModel(torch.nn.Module):
    def __init__(self, num_labels, num_statuses):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        for layer in self.bert.transformer.layer[:3]:
            for param in layer.parameters():
                param.requires_grad = False
        self.dropout = torch.nn.Dropout(0.6)
        self.label_head = torch.nn.Sequential(
            torch.nn.Linear(self.bert.config.hidden_size, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(256, num_labels)
        )
        self.status_head = torch.nn.Sequential(
            torch.nn.Linear(self.bert.config.hidden_size, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(256, num_statuses)
        )

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(out.last_hidden_state[:, 0])
        return self.label_head(pooled), self.status_head(pooled)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChatModerationModel(
    num_labels=len(label_enc.classes_),
    num_statuses=len(status_enc.classes_)
)
model.load_state_dict(torch.load(os.path.join(model_dir, "model_state.pt"), map_location=device))
model.to(device)
model.eval()

# Prediction function
def predict(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)
    
    with torch.no_grad():
        logits_label, logits_status = model(**inputs)
    
    # Get predictions
    lbl_idx = logits_label.argmax(dim=1).item()
    stt_idx = logits_status.argmax(dim=1).item()
    
    predicted_label = label_enc.inverse_transform([lbl_idx])[0]
    predicted_status = status_enc.inverse_transform([stt_idx])[0]
    predicted_decision = status_map[stt_idx]
    
    return predicted_label, predicted_status, predicted_decision

# Process CSV with proper encoding
try:
    # Try UTF-8 first
    df = pd.read_csv("Test_csv.csv", encoding='utf-8')
except UnicodeDecodeError:
    try:
        # Fall back to UTF-16 if UTF-8 fails
        df = pd.read_csv("Test_csv.csv", encoding='utf-16')
    except UnicodeDecodeError:
        # Try other common encodings if needed
        df = pd.read_csv("Test_csv.csv", encoding='latin1')

tqdm.pandas(desc="Processing predictions")

# Run predictions and store results
results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Text']
    try:
        pred_label, pred_status, pred_decision = predict(text)
    except Exception as e:
        print(f"Error processing: {text[:50]}... - {str(e)}")
        pred_label, pred_status, pred_decision = "ERROR", "ERROR", "ERROR"
    
    results.append({
        "Original Text": text,
        "Original Label": row["Label"],
        "Original Status": row["Status"],
        "Original Decision": row["Decision"],
        "Predicted Label": pred_label,
        "Predicted Status": pred_status,
        "Predicted Decision": pred_decision
    })

# Create results dataframe
results_df = pd.DataFrame(results)

# Filter out errors for metrics
error_mask = (results_df["Predicted Label"] == "ERROR") | (results_df["Predicted Status"] == "ERROR")
num_errors = error_mask.sum()
results_df_clean = results_df[~error_mask]

# Calculate accuracy
label_acc = np.mean(results_df_clean["Original Label"] == results_df_clean["Predicted Label"])
status_acc = np.mean(results_df_clean["Original Status"].astype(str) == results_df_clean["Predicted Status"].astype(str))
decision_acc = np.mean(results_df_clean["Original Decision"] == results_df_clean["Predicted Decision"])

print(f"\nEvaluation Metrics:")
print(f"Label Accuracy: {label_acc:.4f}")
print(f"Status Accuracy: {status_acc:.4f}")
print(f"Decision Accuracy: {decision_acc:.4f}")
print(f"Errors: {num_errors}/{len(df)} samples")

# ========================
# Enhanced Reporting
# ========================

# 1. Confusion Matrices
def plot_confusion_matrix(y_true, y_pred, classes, title, filename, figsize=(12, 10)):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix: {title}', fontsize=14)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    print(f"Saved {filename}")

# Get unique classes present in data
label_classes = sorted(set(results_df_clean["Original Label"].unique()).union(
                     set(results_df_clean["Predicted Label"].unique())))
status_classes = sorted(set(results_df_clean["Original Status"].astype(str).unique()).union(
                     set(results_df_clean["Predicted Status"].astype(str).unique())))
decision_classes = ['accepted', 'pending', 'blocked']

# Generate confusion matrices
plot_confusion_matrix(
    results_df_clean["Original Label"],
    results_df_clean["Predicted Label"],
    label_classes,
    "Language Labels",
    "label_confusion_matrix.png"
)

plot_confusion_matrix(
    results_df_clean["Original Status"].astype(str),
    results_df_clean["Predicted Status"].astype(str),
    status_classes,
    "Status Labels",
    "status_confusion_matrix.png",
    figsize=(14, 12)
)

plot_confusion_matrix(
    results_df_clean["Original Decision"],
    results_df_clean["Predicted Decision"],
    decision_classes,
    "Decision Labels",
    "decision_confusion_matrix.png"
)

# 2. Classification Reports
def save_classification_report(y_true, y_pred, classes, title, filename):
    report = classification_report(
        y_true, 
        y_pred, 
        labels=classes,
        target_names=classes, 
        digits=4,
        zero_division=0
    )
    with open(filename, 'w') as f:
        f.write(f"Classification Report: {title}\n")
        f.write("===================================\n\n")
        f.write(report)
    print(f"Saved {filename}")
    return report

# Save reports
label_report = save_classification_report(
    results_df_clean["Original Label"],
    results_df_clean["Predicted Label"],
    label_classes,
    "Language Labels",
    "label_classification_report.txt"
)

status_report = save_classification_report(
    results_df_clean["Original Status"].astype(str),
    results_df_clean["Predicted Status"].astype(str),
    status_classes,
    "Status Labels",
    "status_classification_report.txt"
)

decision_report = save_classification_report(
    results_df_clean["Original Decision"],
    results_df_clean["Predicted Decision"],
    decision_classes,
    "Decision Labels",
    "decision_classification_report.txt"
)

# 3. Error Analysis
error_df = results_df[error_mask]
if not error_df.empty:
    error_df.to_csv("prediction_errors.csv", index=False, encoding='utf-8-sig')
    print(f"Saved error samples to prediction_errors.csv ({len(error_df)} errors)")

# 4. Class-wise Accuracy
def calculate_class_accuracy(df, class_col, pred_col):
    class_acc = {}
    for cls in df[class_col].unique():
        cls_mask = df[class_col] == cls
        if cls_mask.sum() > 0:
            acc = np.mean(df.loc[cls_mask, class_col] == df.loc[cls_mask, pred_col])
            class_acc[cls] = acc
    return class_acc

label_class_acc = calculate_class_accuracy(results_df_clean, "Original Label", "Predicted Label")
status_class_acc = calculate_class_accuracy(results_df_clean, "Original Status", "Predicted Status")
decision_class_acc = calculate_class_accuracy(results_df_clean, "Original Decision", "Predicted Decision")

# 5. Save comprehensive report
with open("full_report.txt", "w", encoding='utf-8') as f:
    f.write("Chat Moderation Model Evaluation Report\n")
    f.write("======================================\n\n")
    f.write(f"Total Samples: {len(df)}\n")
    f.write(f"Errors: {num_errors}\n")
    f.write(f"Label Accuracy: {label_acc:.4f}\n")
    f.write(f"Status Accuracy: {status_acc:.4f}\n")
    f.write(f"Decision Accuracy: {decision_acc:.4f}\n\n")
    
    f.write("\nLabel Class Accuracy:\n")
    for cls, acc in label_class_acc.items():
        f.write(f"  {cls}: {acc:.4f}\n")
    
    f.write("\nStatus Class Accuracy:\n")
    for cls, acc in status_class_acc.items():
        f.write(f"  {cls}: {acc:.4f}\n")
    
    f.write("\nDecision Class Accuracy:\n")
    for cls, acc in decision_class_acc.items():
        f.write(f"  {cls}: {acc:.4f}\n")
    
    f.write("\n\nLabel Classification Report:\n")
    f.write(label_report)
    
    f.write("\n\nStatus Classification Report:\n")
    f.write(status_report)
    
    f.write("\n\nDecision Classification Report:\n")
    f.write(decision_report)

print("\nSaved full_report.txt with comprehensive evaluation metrics")

# Save to CSV with UTF-8 encoding and proper escaping
results_df.to_csv("predictions_report.csv", index=False, encoding='utf-8-sig', escapechar='\\', quoting=1)
print("\nSaved predictions to 'predictions_report.csv'")

c:\Users\Utkarsh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 698274/698274 [43:10<00:00, 269.59it/s]



Evaluation Metrics:
Label Accuracy: 0.9269
Status Accuracy: 0.9193
Decision Accuracy: 0.9247
Errors: 0/698274 samples
Saved label_confusion_matrix.png
Saved status_confusion_matrix.png
Saved decision_confusion_matrix.png
Saved label_classification_report.txt
Saved status_classification_report.txt
Saved decision_classification_report.txt

Saved full_report.txt with comprehensive evaluation metrics

Saved predictions to 'predictions_report.csv'
